In [1]:
# importing libraries
import os
import re
import codecs
import matplotlib.pyplot as plt
import collections
from string import punctuation
import nltk
from nltk.corpus import stopwords
import pandas as pd
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
from nltk.corpus import brown
from nltk.corpus import reuters
from nltk.corpus import webtext
from nltk.corpus import gutenberg

words_r = reuters.words()
words_b = brown.words()
words_w = webtext.words()
words_g = gutenberg.words()

brown_words = dict(collections.Counter([lemmatizer.lemmatize(i.lower()) for i in words_b]))
reuters_words = dict(collections.Counter([lemmatizer.lemmatize(i.lower()) for i in words_r]))
web_words = dict(collections.Counter([lemmatizer.lemmatize(i.lower()) for i in words_w]))
guten_words = dict(collections.Counter([lemmatizer.lemmatize(i.lower()) for i in words_g]))

import spacy
nlp = spacy.load("en_core_web_sm")
stop = set(stopwords.words('english'))

%matplotlib inline

ImportError: No module named spacy

In [ ]:
stop_brown = [i[0] for i in sorted(brown_words.items(), key=lambda k: k[1], reverse=True) if i[1] > 100]
stop_reuters = [i[0] for i in sorted(reuters_words.items(), key=lambda k: k[1], reverse=True) if i[1] > 250]
stop_web = [i[0] for i in sorted(web_words.items(), key=lambda k: k[1], reverse=True) if i[1] > 50]
stop_guten = [i[0] for i in sorted(guten_words.items(), key=lambda k: k[1], reverse=True) if i[1] > 100]

len(stop_brown), len(stop_reuters), len(stop_web), len(stop_guten)

In [ ]:
BASE_DIR = 'data'
DATA_DIR = codecs.open(os.path.join(BASE_DIR, 'books/HP1.txt'), 'rb', encoding='utf-8').readlines()
COMMON_DIR = codecs.open(os.path.join(BASE_DIR, 'google_10000.txt'), 'rb', encoding='utf-8').readlines()
true_data = pd.read_csv(os.path.join(BASE_DIR, 'ground_truth/HP1.csv'), sep='\t')
stop_words = codecs.open(os.path.join(BASE_DIR, 'stop.txt'), 'rb', encoding='utf-8').readlines()

In [ ]:
stop.update(list(set([i.strip().lower() for i in stop_words])))
stop.update(list(set([i.strip().lower() for i in stop_brown])))
stop.update(list(set([i.strip().lower() for i in stop_web])))
stop.update(list(set([i.strip().lower() for i in stop_reuters])))
stop.update(list(set([i.strip().lower() for i in stop_guten])))

In [ ]:
len(set(stop))

In [ ]:
true_data_lst = true_data.iloc[:, 0].tolist()

In [ ]:
len(true_data_lst)

In [15]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }
punctuation = "'!()-[]{};:'\,<>./?@#$%^&*_~"

def expand_contractions(data_str):
    specials = ["’", "‘", "´", "`", "’"]
    for s in specials:
        data_str = data_str.replace(s, "'")
    data_str = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in data_str.split(" ")])
    return data_str

def extra_spaces(data_str):
    return re.sub(r'\s{1,}', ' ', data_str)

def punctuations(data_str):
    data_str = data_str.replace("'s", "")
    for x in data_str.lower():
        if x in punctuation: 
            data_str = data_str.replace(x, "")
    return data_str

def remove_stop(data_str):
    word_tokens = nltk.word_tokenize(data_str)
    filtered_sentence = [w for w in word_tokens if not w in stop] 
    return ' '.join(filtered_sentence)

def preprocess(data_str):
    data_str = expand_contractions(data_str)
    data_str = remove_stop(data_str)
    data_str = punctuations(data_str)
    data_str = extra_spaces(data_str)
    return data_str

def precision(pred, actual):
    N = [i for i in pred if i in actual]
    _D = [i for i in pred if i not in actual]
    return len(N) / (len(N)+len(_D)), N, _D

def recall(pred, actual):
    N = [i for i in pred if i in actual]
    _D = [i for i in actual if i not in pred]
    return len(N) / (len(N)+len(_D)), N, _D

def f1score(precision, recall):
    return (2*precision*recall) / (precision + recall)

def ner(data):
    ner_chunks = set()
    ner_chunks_cat = set()
    doc = nlp(data)
    for ent in doc.ents:
        ner_chunks.add(ent.text)
        ner_chunks_cat.add(ent.label_)
    return ner_chunks, ner_chunks_cat

full_data = ' '.join([i.strip() for i in DATA_DIR if len(i.strip())>1])
# ner_chunks, ner_chunks_cat = ner(full_data)
# for i in ner_chunks:
#     full_data = full_data.replace(i, '')
full_data_ = full_data.lower()
data = nltk.sent_tokenize(full_data_)
#data = [i for i in data]

In [16]:
sss = set([i for i in true_data_lst if i in ' '.join(data).split()])

In [37]:
# from gensim.summarization import keywords
# np_chunkss = keywords(' '.join(data), lemmatize=True).split('\n')

In [38]:
# len(np_chunkss)

635

In [49]:
# #####
# # np_chunks = set()
# # doc = nlp(data)
# # for chunk in doc.noun_chunks:
# #     np_chunks.add(chunk.text)

# ######
# #np_chunks = set(nltk.word_tokenize(data))

# ######
np_chunks = []
is_noun = lambda pos: pos[:2] in ['NN', 'JJ']
for sent in data:
    tokenized = nltk.word_tokenize(sent)
    np_chunks.append([word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)])

In [50]:
np_chunks = set([lemmatizer.lemmatize(item.strip(punctuation)) for sublist in np_chunks for item in sublist])
np_chunks = [i for i in np_chunks if '-' not in i]

In [51]:
# np_chunks = set([lemmatizer.lemmatize(item.strip(punctuation)) for item in ['sense']])


In [52]:
# np_chunks.extend(np_chunkss)

In [53]:
np_chunks = set(np_chunks)

In [54]:
prec, TP, FP = precision(np_chunks, true_data_lst)
rec, TP, FN = recall(np_chunks, true_data_lst)

print (f"Precision = {prec}")
print (f"Recall = {rec}")
print (f"F1 = {f1score(prec, rec)}")
print (f"TP={len(TP)}, FP={len(FP)}, FN={len(FN)}")


Precision = 0.020177562550443905
Recall = 0.7352941176470589
F1 = 0.03927729772191673
TP=75, FP=3642, FN=27


In [71]:
def preprocess(np_chunks):
    keep, remove = [], []
    for i in np_chunks:
        if i not in stop and '-' not in i and len(i) > 3:
            keep.append(i)
        else:
            remove.append(i)
    return keep, remove

data_preprocess_removed, preprocess_removed = preprocess(np_chunks)

In [72]:
len(data_preprocess_removed), len(preprocess_removed)

(2096, 1328)

In [73]:
prec, TP, FP = precision(data_preprocess_removed, true_data_lst)
rec, TP, FN = recall(data_preprocess_removed, true_data_lst)

print (f"Precision = {prec}")
print (f"Recall = {rec}")
print (f"F1 = {f1score(prec, rec)}")
print (f"TP={len(TP)}, FP={len(FP)}, FN={len(FN)}")

Precision = 0.03482824427480916
Recall = 0.7156862745098039
F1 = 0.06642402183803459
TP=73, FP=2023, FN=29


In [74]:
common = [i.strip() for i in COMMON_DIR]

def remove_google_common(np_chunks):
    keep, remove = [], []
    for i in np_chunks:
        if i not in common:
            keep.append(i)
        else:
            remove.append(i)
            
    return keep, remove

data_common_removed, common_removed = remove_google_common(data_preprocess_removed)

In [75]:
len(data_common_removed), len(common_removed)

(1413, 683)

In [76]:
prec, TP, FP = precision(data_common_removed, true_data_lst)
rec, TP, FN = recall(data_common_removed, true_data_lst)

print (f"Precision = {prec}")
print (f"Recall = {rec}")
print (f"F1 = {f1score(prec, rec)}")
print (f"TP={len(TP)}, FP={len(FP)}, FN={len(FN)}")

Precision = 0.05024769992922859
Recall = 0.696078431372549
F1 = 0.09372937293729372
TP=71, FP=1342, FN=31


In [79]:
DDD = nltk.sent_tokenize(full_data)


In [110]:
ner_chunks = set()
ner_chunks_cat = set()
for sent in DDD:
    doc = nlp(sent)
    for ent in doc.ents:
        ner_chunks.add(ent.text)
        ner_chunks_cat.add(ent.label_)

In [86]:
len(ner_chunks)

1261

In [112]:
ner_chunks = [i.lower() for i in ner_chunks]

In [134]:
def remove_ner(np_chunks):
    keep, remove = [], []
    for i in np_chunks:
        if i not in sss:
            keep.append(i)
        else:
            remove.append(i)
    return keep, remove

data_ner_removed, ner_removed = remove_ner(data_common_removed)

In [135]:
len(data_ner_removed), len(ner_removed)

(145, 1268)

In [136]:
# ner_removed

In [137]:
prec, TP, FP = precision(data_ner_removed, true_data_lst)
rec, TP, FN = recall(data_ner_removed, true_data_lst)

print (f"Precision = {prec}")
print (f"Recall = {rec}")
print (f"F1 = {f1score(prec, rec)}")
print (f"TP={len(TP)}, FP={len(FP)}, FN={len(FN)}")

Precision = 0.07586206896551724
Recall = 0.10784313725490197
F1 = 0.08906882591093118
TP=11, FP=134, FN=91


In [97]:
# wc

In [98]:
wc = dict(collections.Counter(nltk.word_tokenize(' '.join([lemmatizer.lemmatize(i).strip(punctuation) for i in ' '.join(data).split()]))))
# maxx = sorted(wc.items(), key=lambda k: k[1], reverse=True)[0][1]
# wc_freq = {k: v/maxx for k, v in wc.items()}

# plt.bar(range(len(wc_freq)), list(wc_freq.values()), align='center')
# plt.xticks(range(len(wc_freq)), list(wc_freq.keys()))

# plt.show()

In [99]:
# def remove_high_low_freq(np_chunks):
#     keep, remove = [], []
#     for i in np_chunks:
#         if i in wc_freq:
#             if wc_freq[i] < 0.95:
#                 keep.append(i)
#             else:
#                 remove.append(i)
#         else:
#             remove.append(i)
#     return keep, remove


# data_freq_removed, freq_removed = remove_high_low_freq(data_preprocess_removed)

In [100]:
# for i in data_freq_removed:
#     if '-' in i:
#         print (i)

In [101]:
# len(data_freq_removed), len(freq_removed)

In [102]:
# prec, TP, FP = precision(data_freq_removed, true_data_lst)
# rec, TP, FN = recall(data_freq_removed, true_data_lst)

# print (f"Precision = {prec}")
# print (f"Recall = {rec}")
# print (f"F1 = {f1score(prec, rec)}")
# print (f"TP={len(TP)}, FP={len(FP)}, FN={len(FN)}")

In [103]:
def remove_repeated_noisy_chars(np_chunks):
    keep, remove = [], []
    for i in np_chunks:
        if len(re.findall(r'(.)\1{2,}', i)):
            remove.append(i)
        else:
            keep.append(i)
    return keep, remove

data_noisy_removed, noisy_removed = remove_repeated_noisy_chars(data_ner_removed)

In [104]:
len(data_noisy_removed), len(noisy_removed)

(1162, 7)

In [105]:
noisy_removed

['haaa', 'hmmm', 'shhhh', 'haaaaaa', 'aaaargh', 'aaaaaaaaaaargh', 'shhh']

In [106]:
prec, TP, FP = precision(data_noisy_removed, true_data_lst)
rec, TP, FN = recall(data_noisy_removed, true_data_lst)

print (f"Precision = {prec}")
print (f"Recall = {rec}")
print (f"F1 = {f1score(prec, rec)}")
print (f"TP={len(TP)}, FP={len(FP)}, FN={len(FN)}")

Precision = 0.043029259896729774
Recall = 0.49019607843137253
F1 = 0.07911392405063292
TP=50, FP=1112, FN=52


In [107]:
def remove_specific_corpus(np_chunks):
    keep, remove = [], []
    for i in np_chunks:
        if i in wc:
            c1 = wc[i]
            if i in brown_words:
                c2 = brown_words[i]
                if c1 > c2:
                    keep.append(i)
                else: remove.append(i)
            else:
                keep.append(i)
        else:
            keep.append(i)
            
    return keep, remove

data_specific_removed, specific_removed = remove_specific_corpus(data_noisy_removed)

In [108]:
len(data_specific_removed), len(specific_removed)

(467, 695)

In [109]:
prec, TP, FP = precision(data_specific_removed, true_data_lst)
rec, TP, FN = recall(data_specific_removed, true_data_lst)

print (f"Precision = {prec}")
print (f"Recall = {rec}")
print (f"F1 = {f1score(prec, rec)}")
print (f"TP={len(TP)}, FP={len(FP)}, FN={len(FN)}")

Precision = 0.06638115631691649
Recall = 0.30392156862745096
F1 = 0.10896309314586994
TP=31, FP=436, FN=71


In [1]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.keyedvectors import KeyedVectors
import numpy as np

filepath = "/home/prakhar/Downloads/GoogleNews-vectors-negative300.bin"

from gensim.models import KeyedVectors
wv_from_bin = KeyedVectors.load_word2vec_format(filepath, binary=True) 
#extracting words7 vectors from google news vector
embeddings_index = {}
for word, vector in zip(wv_from_bin.vocab, wv_from_bin.vectors):
    coefs = np.asarray(vector, dtype='float32')
    embeddings_index[word] = coefs
    

In [438]:
def avg_feature_vector(sentence, model, num_features):
    words = sentence.split()
    #feature vector is initialized as an empty array
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in embeddings_index.keys():
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [439]:
context_vec = avg_feature_vector(full_data, model= embeddings_index, num_features=300)

In [440]:
import pandas as pd
from scipy.spatial import distance

d = pd.DataFrame(data_specific_removed, columns=['glossary'])
d['sim'] = d['glossary'].apply(lambda x: distance.cosine(avg_feature_vector(x, model= embeddings_index, num_features=300), context_vec))

In [441]:
d.shape

(878, 2)

In [445]:
dd = d.sort_values('sim', ascending=False).head(800)

In [446]:
dd.shape

(800, 2)

In [447]:
prec, TP, FP = precision(dd['glossary'].tolist(), true_data_lst)
rec, TP, FN = recall(dd['glossary'].tolist(), true_data_lst)

print (f"Precision = {prec}")
print (f"Recall = {rec}")
print (f"F1 = {f1score(prec, rec)}")
print (f"TP={len(TP)}, FP={len(FP)}, FN={len(FN)}")

Precision = 0.06125
Recall = 0.4803921568627451
F1 = 0.10864745011086474
TP=49, FP=751, FN=53


In [448]:
dd['glossary'].tolist()

['stra',
 'argus',
 'infusion',
 'mortis',
 'multilevel',
 'nimbus',
 'discus',
 'boater',
 'ember',
 'vein',
 'ensnaring',
 'heartstrings',
 'headmistress',
 'patil',
 'headmaster',
 'hygienic',
 'sens',
 'stalagmite',
 'trevor',
 'sleepiness',
 'javelin',
 'emporium',
 'smelting',
 'pigtail',
 'locomotor',
 'newscaster',
 'eyeglass',
 'coil',
 'weatherman',
 'parkinson',
 'baron',
 'fastening',
 'stargazer',
 'palomino',
 'detention',
 'wormwood',
 'pupil',
 'wastepaper',
 'cleansweep',
 'supple',
 'albus',
 'ruff',
 'tinned',
 'passageway',
 'phial',
 'sheared',
 'fungi',
 'pellet',
 'tendril',
 'fang',
 'warty',
 'squid',
 'tabby',
 'witchcraft',
 'friar',
 'unseated',
 'goshawk',
 'marge',
 'dolphin',
 'spying',
 'beetle',
 'aconite',
 'elixir',
 'transfiguration',
 'seared',
 'gorgon',
 'teabags',
 'defrosting',
 'snout',
 'quaffle',
 'bluebell',
 'bott',
 'bowler',
 'wizarding',
 'escalator',
 'archway',
 'warlock',
 'fingertip',
 'peppermint',
 'clinking',
 'chessboard',
 'mida